In [15]:
import pandas as pd
import numpy as np
import psycopg2

In [2]:
# Connect to PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="Root@123",
)

In [3]:
# Define a function to execute SQL queries and return results as a DataFrame
def execute_query(query):
    with conn.cursor() as cursor:
        cursor.execute(query)
        columns = [desc[0] for desc in cursor.description]
        return pd.DataFrame(cursor.fetchall(), columns=columns)

In [4]:
query = """
SELECT * FROM public.xdr_data
"""

In [5]:
# Execute query and load data into DataFrame
df = execute_query(query)

In [6]:
# Check the first few rows of the DataFrame
print(df.head())

      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...          20247395.0          19111729.0   

In [12]:
def aggregate_user_behavior(df):
    """
    Aggregate user behavior metrics per user (MSISDN).

    Args:
    - df (DataFrame): Input DataFrame containing the raw data.

    Returns:
    - user_behavior_df (DataFrame): DataFrame with aggregated user behavior metrics.
    """
    # Group by MSISDN and aggregate metrics
    user_behavior_df = (
        df.groupby("MSISDN/Number")
        .agg(
            {
                "Bearer Id": "count",
                "Dur. (ms)": "sum",
                "Total DL (Bytes)": "sum",
                "Total UL (Bytes)": "sum",
                "Social Media DL (Bytes)": "sum",
                "Social Media UL (Bytes)": "sum",
                "Google DL (Bytes)": "sum",
                "Google UL (Bytes)": "sum",
                "Email DL (Bytes)": "sum",
                "Email UL (Bytes)": "sum",
                "Youtube DL (Bytes)": "sum",
                "Youtube UL (Bytes)": "sum",
                "Netflix DL (Bytes)": "sum",
                "Netflix UL (Bytes)": "sum",
                "Gaming DL (Bytes)": "sum",
                "Gaming UL (Bytes)": "sum",
                "Other DL (Bytes)": "sum",
                "Other UL (Bytes)": "sum",
            }
        )
        .reset_index()
    )

    return user_behavior_df

In [13]:
def handle_missing_values_and_outliers(df):
    """
    Handle missing values and outliers in the DataFrame by replacing them with column mean.

    Args:
    - df (DataFrame): Input DataFrame containing the raw data.

    Returns:
    - cleaned_df (DataFrame): DataFrame with missing values and outliers replaced with column mean.
    """
    # Replace missing values with column mean
    cleaned_df = df.fillna(df.mean())

    # Replace outliers with column mean
    for column in df.columns:
        if df[column].dtype != "object":
            cleaned_df[column] = np.where(
                (cleaned_df[column] < cleaned_df[column].quantile(0.05))
                | (cleaned_df[column] > cleaned_df[column].quantile(0.95)),
                cleaned_df[column].mean(),
                cleaned_df[column],
            )

    return cleaned_df

In [16]:
# Aggregate user behavior metrics
user_behavior_df = aggregate_user_behavior(df)

# Handle missing values and outliers
cleaned_df = handle_missing_values_and_outliers(user_behavior_df)

# Display the first few rows of the cleaned DataFrame
print(cleaned_df.head())

   MSISDN/Number  Bearer Id  Dur. (ms)  Total DL (Bytes)  Total UL (Bytes)  \
0   4.511474e+10        1.0   116720.0      8.426375e+08      3.605311e+07   
1   4.511474e+10        1.0   181230.0      1.207552e+08      3.610446e+07   
2   4.511474e+10        1.0   134969.0      5.566597e+08      3.930682e+07   
3   4.511474e+10        1.0    49878.0      4.019932e+08      5.730942e+07   
4   4.511474e+10        2.0    37104.0      1.363130e+09      9.428053e+07   

   Social Media DL (Bytes)  Social Media UL (Bytes)  Google DL (Bytes)  \
0                2206504.0                  25631.0          3337123.0   
1                2598548.0                  62017.0          4197697.0   
2                3148004.0                  47619.0          3343483.0   
3                 251469.0                  28825.0          5937765.0   
4                2861230.0                  51312.0         13728668.0   

   Google UL (Bytes)  Email DL (Bytes)  Email UL (Bytes)  Youtube DL (Bytes)  \
0     